# **빅데이터 개론 Lab 13 - 랜덤 포레스트(Random Forests)**

참고자료 : https://www.notion.so/TA-2689a38b5289413a82671d3956fea103

- - -




### **<앙상블 모델 (Ensemble model)>**

  * 앙상블(ensemble) 모델은 여러 개의 분류 모델에 의한 결과를 종합하여 분류의 정확도를 높이는 방법이다.
  * 여러 개의 모델을 합쳐 일반화 성능 향상 : 대중의 지혜(Wisdom of Crowd)
  * 표본추출법으로 원래 데이터에서 여러 개의 훈련용 데이터 집합을 만들어 각각의 데이터 집합에서 하나의 분류기를 만들어 앙상블 하는 방법이다. 
  * 새로운 자료에 대해 각 분류기의 예측값들의 가중 투표(weighted vote) 등을 통해 분류를 수행한다.

<br>

<p align="center"><img src="https://github.com/Jin0331/TA/blob/master/image/ensemble_1.png?raw=true" width="425"/> <img src="https://github.com/Jin0331/TA/blob/master/image/ensemble_2.png?raw=true" width="425"/></p>


* **개별 모형과 비교한 앙상블 모델의 장점**
  * 치우침이 있는 여러 모형의 평균을 취하면, 어느 쪽에도 치우치지 않는 결과(평균)를 얻게 된다.
  * 분산을 감소시킨다: 한 개 모형으로부터의 단일 의견보다 여러 모형의 의견을 결합하면 변동이 작아진다.
  * 과적합의 가능성을 줄여준다: 각 모형으로부터 예측을 결합하면 과적합의 여지가 줄어든다.

* **앙상블 방법**

  * **배깅 (bagging) : 중복을 허용하는 샘플링**
  * **부스팅(boosting) : 이전 예측기의 오차를 보완해서 샘플링**
  * ~~페이스팅(pasting) : 중복을 허용하지 않는 샘플링~~
  * ~~스태킹(stacking) : 앙상블 결과 위에 예측을 위한 모델 추가~~

In [ ]:
install.packages(c("tidyverse", "caret", "e1071", "ipred","randomForest"))

### **앙상블 모델과 비교하기 위한 Decision Tree 생성 - Heart Disease Data이용** 

https://archive.ics.uci.edu/ml/datasets/heart+disease

* 변수 설명

```
Age : age in years
Sex: sex (1 = male; 0 = female) # Factor
ChestPain : (typical angina, atypical angina, non-anginal pain, asymptomatic # Factor
RestBP(혈압) : resting blood pressure
Chol(콜레스테롤 수치) : serum cholestoral in mg/dl
Fbs(혈당) : (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) # Factor
Restecg(심전도) : (0 = normal, 1 = having ST-T wave abnormality, 2 =  showing probable or definite left ventricular hypertrophy by Estes' criteria) # Factor
MaxHR : maximum heart rate achieved
ExAng(협심증?): exercise induced angina (1 = yes; 0 = no) # Factor
Oldpeak = ST depression induced by exercise relative to rest
Slope: the slope of the peak exercise ST segment(1 = upsloping, 2 = flat, 3 = downsloping) # Factor
Ca: number of major vessels (0-3) colored by flourosopy # Factor
Thal: 3 = normal; 6 = fixed defect; 7 = reversable defect # Factor

# the predicted attribute(반응변수)

AHD : diagnosis of heart disease (angiographic disease status)(0 = < 50% diameter narrowing, 1 =  > 50% diameter narrowing)

# http://archive.ics.uci.edu/ml/datasets/heart+Disease
```


In [ ]:
library(tidyverse)
heart_df <- read_csv("https://raw.githubusercontent.com/Jin0331/TA/master/data/heart/Heart.csv") 
str(heart_df)

* mutate를 이용한 데이터 타입 변경(int or chr ---> factor)

In [ ]:
heart_df <- heart_df %>% 
 mutate_at(`.vars` = c("Sex", "ChestPain", "Fbs", "RestECG", "ExAng", "Slope", "Ca", "Thal", "AHD"), `.funs` = as.factor)
heart_df %>% str()

* **train-test split**

In [ ]:
library(caret) 
set.seed(31)
index <- createDataPartition(y = heart_df$AHD, p = 0.7, list = FALSE) 
train <- heart_df[index, ]
test <- heart_df[-index, ]

* Decision Tree Model 생성

In [ ]:
library(rpart)
AHD_detection <- rpart(formula = AHD ~ ., data = train, method = "class")

  - xerror(cross validation error)가 최소가 되는 CP를 선택

In [ ]:
min_xerror_cp <- AHD_detection$cptable %>% as_tibble() %>%
  filter(xerror == min(xerror)) %>% pull(CP)

In [ ]:
min_xerror_cp

* prune

In [ ]:
AHD_detection_pr <- rpart::prune(AHD_detection, cp = min_xerror_cp)

* **test를 이용한 예측 및 평가**

In [ ]:
test %>% show()

In [ ]:
predict_value <- predict(AHD_detection_pr, test, type = "class") %>% 
 tibble(predict_value = .)
predict_check <- test %>% select(AHD) %>% dplyr::bind_cols(., predict_value) 

In [ ]:
cm <- caret::confusionMatrix(predict_value$predict_value, test$AHD)
draw_confusion_matrix(cm)

#### **배깅(Bagging)**

* 배깅(bagging) : ``중복``을 허용하는 샘플링. Bootstrap Aggregation의 줄임말
  * *통계학에서는 중복을 허용하는 샘플링을 부트스트래핑(bootstrapping)이라고 함*

* 배깅은 중복 추출 방법을 사용하기 때문에 같은 데이터가 한 데이터셋에  여러 번 추출될 수도 있고, 어떤 데이터는 추출되지 않을 수도 있다.

* 데이터가 충분히 큰 경우, 각 데이터가 하나의 붓스트랩 표본에서 제외될 확률은 36.78%이다

<br>

<p align="center"><img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbTZVqu%2Fbtqw4oKck4I%2FKbO0ih5GDB5qP2HNtUsGa0%2Fimg.png" width="600"/> 

<p align="center"><img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FReb8Q%2Fbtqw5gkMnjn%2FaWVHsgQihxY7wmaNEN8Ay0%2Fimg.jpg" width="600"/> 

* bagging

<p align="center"><img src="https://github.com/Jin0331/TA/blob/master/image/bagging.png?raw=true" width="600"/></p>


In [ ]:
library(ipred)

In [ ]:
set.seed(66)
AHD_bagging <- ipred::bagging(AHD ~ ., data = train, nbagg = 100)

In [ ]:
predict_value <- predict(AHD_bagging, test, type = "class") %>% 
 tibble(predict_value = .)
predict_check <- test %>% select(AHD) %>% dplyr::bind_cols(., predict_value) 

In [ ]:
cm <- caret::confusionMatrix(predict_value$predict_value, test$AHD)
draw_confusion_matrix(cm)

#### **부스팅(Boosting)**

* 부스팅(Boosting) : 약한 학습기를 여러 개 연결해서 강한 학습기를 만드는 앙상블 방법
   * 앞 모델을 보완해 나가도록 모델을 학습하여 연결해 나감
   * AdaBoost (Adpative Boosting), Gradient Boosting 이 있음

<br>

<p align="center"><img src="https://github.com/Jin0331/TA/blob/master/image/boosting.png?raw=true" width="450"/></p>

In [ ]:
install.packages("adabag")
library(adabag)

In [ ]:
# Facotr 변수 존재하면 적용 안 되는 듯..
AHD_boost <- adabag::boosting(AHD ~ ., data = train)

### **<랜덤 포레스트(Random Forests)>**

* 랜덤포레스트(random forest)는 배깅에 랜덤과정(설명변수)을 추가한 방법이다.
* 원래 트레이닝  자료로부터 붓스트랩 샘플을 추출하고, 각 붓스트랩 샘플에 대해 트리를 형성해 나가는 과정은 배깅과 유사하나, 각 노드마다 모든 예측변수 안에서 최적의 분할(split)을 선택하는 방법 대신 예측변수들을 임의로 추출하고, 추출된 변수 내에서 최적의 분할을 만들어 나가는 방법을 사용한다.
* 새로운 자료에 대한 예측은 분류(classification)의 경우는 다수결(majority votes)로, 회귀(regression)의 경우에는 평균을 취하는 방법을 사용한다.

<br>

<p align="center"><img src="https://github.com/Jin0331/TA/blob/master/image/random_forests.jpg?raw=true" width="550"></p>


* __배깅과 랜덤 포레스트의 차이?__
  * 배깅의 일종. 배깅과 다른 점은, ``'설명변수'도 무작위로 선택``. 즉, 설명변수를 무작위로 선택함으로써, 트리의 다양성을 확보하여 모형간의 상관관계를 줄이고자 하는 것



In [ ]:
#install.packages("randomForest")
library(randomForest)

In [ ]:
summary(train)

In [ ]:
AHD_RF <- randomForest(AHD ~ . -X1, data = train, na.action = na.omit, importance = T, mtry = 7, ntree = 1000)
AHD_RF

* mtry 골드 스탠다드(gold standard) : sqrt(설명변수 개수)

In [ ]:
mtry_sqrt <- train %>% select(-X1, -AHD) %>% colnames() %>%
 length() %>% sqrt() %>% floor()

mtry_sqrt

In [ ]:
set.seed(52)
AHD_RF <- randomForest(AHD ~ . -X1, data = train, na.action = na.omit, importance = T, mtry = mtry_sqrt, ntree = 1000)
AHD_RF

In [ ]:
varImpPlot(AHD_RF, type=2, pch=19, col=1, cex=1, main="")

In [ ]:
plot(AHD_RF)
legend(x = 750, y = 0.34, c("no", "yes", "mean"), col = c("pink", "green", "black"), pch = c(1,1,1), cex = 1.3)

* 예측 및 평가

In [ ]:
predict_value <- predict(AHD_RF, test, type = "class") %>% 
 tibble(predict_value = .)
predict_check <- test %>% select(AHD) %>% dplyr::bind_cols(., predict_value) 

In [ ]:
cm <- caret::confusionMatrix(predict_value$predict_value, test$AHD)
draw_confusion_matrix(cm)

- - -

#### **B. titanic**

* https://www.kaggle.com/c/titanic/data

**<kaggle의 타이타닉 data>**

  * survived : 생존=1, 죽음=0
  * pclass : 승객 등급. 1등급=1, 2등급=2, 3등급=3
  * sibsp : 함께 탑승한 형제 또는 배우자 수
  * parch : 함께 탑승한 부모 또는 자녀 수
  * ticket : 티켓 번호
  * cabin : 선실 번호
  * embarked : 탑승장소 S=Southhampton, C=Cherbourg, Q=Queenstown

In [ ]:
train <- read_csv("https://raw.githubusercontent.com/Jin0331/TA/master/data/titanic/train.csv")

In [ ]:
str(train)

In [ ]:
train %>% summary()

* 범주형 변수 확인

In [ ]:
train <- train %>% 
 select(-PassengerId, -Name, -Cabin, -Ticket) %>% mutate_at(c("Survived","Sex","Embarked", "Pclass"), factor)
summary(train)

* Hmisc::impute을 이용한 NA 값 대체(평균, 중앙값, 특정 숫자)

* https://m.blog.naver.com/PostView.nhn?blogId=tjdudwo93&logNo=221142961499&proxyReferer=https:%2F%2Fwww.google.com%2F

In [ ]:
install.packages("Hmisc")

In [ ]:
library(Hmisc)
train$Age <- impute(train$Age, median)

In [ ]:
train %>% summary()

In [ ]:
train <- train %>% na.omit()

In [ ]:
train %>% summary()

* **train을 이용한 Random Forests 모델 생성**

In [ ]:
mtry_sqrt <- train %>% select(-Survived) %>% colnames() %>%
 length() %>% sqrt() %>% floor()
mtry_sqrt

In [ ]:
# set.seed(29)
model <- randomForest(Survived ~ ., data = train,importance = T, mtry = mtry_sqrt, ntree = 1000)
model

* caret과 e1071 패키지를 이용한 grid search

https://www.guru99.com/r-random-forest-tutorial.html

* Default setting : K-fold cross validation is controlled by the trainControl() function

```
trainControl(method = "cv", number = n, search ="grid")
arguments
- method = "cv": The method used to resample the dataset. 
- number = n: Number of folders to create
- search = "grid": Use the search grid method. For randomized method, use "grid"
Note: You can refer to the vignette to see the other arguments of the function.
```

In [ ]:
# Define the control
trControl <- trainControl(method = "cv", number = 10, search = "grid")

* Default setting:he build the model with the default values
  * parameter를 따로 지정하지 않는다면, default로 mtry에 대한 grid search

In [ ]:
# Run the model
rf_default <- train(Survived~., data = train, method = "rf", metric = "Accuracy", trControl = trControl)
# Print the results
print(rf_default)

* Search best mtry

In [ ]:
tuneGrid <- expand.grid(.mtry = c(2: 6)) # 변수개수 7개. 
rf_mtry <- train(Survived~.,
    data = train, method = "rf",
    metric = "Accuracy", tuneGrid = tuneGrid,
    trControl = trControl, importance = T, ntree = 1000)
print(rf_mtry)

In [ ]:
best_mtry <- rf_mtry$bestTune$mtry
best_mtry

* best ntree

In [ ]:
tuneGrid <- expand.grid(.mtry = best_mtry)
store_maxtrees <- list()
for (ntree in c(250, 300, 350, 400, 450, 500, 550, 600, 800, 1000, 2000)) {
    set.seed(51)
    rf_maxtrees <- train(Survived~.,
        data = train,
        method = "rf",
        metric = "Accuracy",
        tuneGrid = tuneGrid,
        trControl = trControl,
        importance = T,
        ntree = ntree)
    key <- toString(ntree)
    store_maxtrees[[key]] <- rf_maxtrees
}
results_tree <- resamples(store_maxtrees)
summary(results_tree)

best parameter
* mtry = 4
* ntree = 300

In [ ]:
set.seed(8)
model_grid <- randomForest(Survived ~ ., data = train, importance = T, mtry = 4, ntree = 300)
model_grid

In [ ]:
plot(model_grid)
legend(x = 200, y = 0.35, c("Survived:yes(1)", "Survived:no(0)", "mean"), col = c("green", "pink", "black"), pch = c(1,1,1), cex = 1.3)

* 생성한 2개의 Decision Tree 모델을 이용하여 kaggle에 제출 및 평가받기

In [ ]:
test <- read_csv("https://raw.githubusercontent.com/Jin0331/TA/master/data/titanic/test.csv")
test %>% summary()

* NA 값 추정(median)

In [ ]:
test$Age <- impute(test$Age, median)
test$Fare <- impute(test$Age, median)
test %>% summary()

* 범주형 변수

In [ ]:
test <- test %>% 
 select(-Name, -Cabin, -Ticket) %>% mutate_at(c("Sex","Embarked", "Pclass"), factor)
summary(test)

* 예측(model, model_grid 모델)

In [ ]:
# model
predict_value <- predict(model, test, type = "class") %>% tibble(Survived = .)
submission1 <- test %>% select(PassengerId) %>% dplyr::bind_cols(., predict_value)

# model_grid
predict_value <- predict(model_grid, test, type = "class") %>% tibble(Survived = .)
submission2 <- test %>% select(PassengerId) %>% dplyr::bind_cols(., predict_value)

In [ ]:
submission1 %>% head(20)

In [ ]:
submission2 %>% head(20)

In [ ]:
 # write
 submission1 %>% write_csv(path = "submission1.csv")
 submission2 %>% write_csv(path = "submission2_grid.csv")

In [ ]:
model_bagg <- ipred::bagging(Survived ~ ., data = train, nbagg = 200)

In [ ]:
#@title
predict_value <- predict(model_bagg, test, type = "class") %>% tibble(Survived = .)
submission1 <- test %>% select(PassengerId) %>% dplyr::bind_cols(., predict_value) %>%
 write_csv(path = "submission3_bagging.csv")

### Confusion Matrix plot code

In [ ]:
#https://stackoverflow.com/questions/23891140/r-how-to-visualize-confusion-matrix-using-the-caret-package

draw_confusion_matrix <- function(cm) {

  total <- sum(cm$table)
  res <- as.numeric(cm$table)

  # Generate color gradients. Palettes come from RColorBrewer.
  greenPalette <- c("#F7FCF5","#E5F5E0","#C7E9C0","#A1D99B","#74C476","#41AB5D","#238B45","#006D2C","#00441B")
  redPalette <- c("#FFF5F0","#FEE0D2","#FCBBA1","#FC9272","#FB6A4A","#EF3B2C","#CB181D","#A50F15","#67000D")
  getColor <- function (greenOrRed = "green", amount = 0) {
    if (amount == 0)
      return("#FFFFFF")
    palette <- greenPalette
    if (greenOrRed == "red")
      palette <- redPalette
    colorRampPalette(palette)(100)[10 + ceiling(90 * amount / total)]
  }

  # set the basic layout
  layout(matrix(c(1,1,2)))
  par(mar=c(2,2,2,2))
  plot(c(100, 345), c(300, 450), type = "n", xlab="", ylab="", xaxt='n', yaxt='n')
  title('CONFUSION MATRIX', cex.main=2)

  # create the matrix 
  classes = colnames(cm$table)
  rect(150, 430, 240, 370, col=getColor("green", res[1]))
  text(195, 435, classes[1], cex=1.2)
  rect(250, 430, 340, 370, col=getColor("red", res[3]))
  text(295, 435, classes[2], cex=1.2)
  text(125, 370, 'Predicted', cex=1.3, srt=90, font=2)
  text(245, 450, 'Actual', cex=1.3, font=2)
  rect(150, 305, 240, 365, col=getColor("red", res[2]))
  rect(250, 305, 340, 365, col=getColor("green", res[4]))
  text(140, 400, classes[1], cex=1.2, srt=90)
  text(140, 335, classes[2], cex=1.2, srt=90)

  # add in the cm results
  text(195, 400, res[1], cex=1.6, font=2, col='white')
  text(195, 335, res[2], cex=1.6, font=2, col='white')
  text(295, 400, res[3], cex=1.6, font=2, col='white')
  text(295, 335, res[4], cex=1.6, font=2, col='white')

  # add in the specifics 
  plot(c(100, 0), c(100, 0), type = "n", xlab="", ylab="", main = "DETAILS", xaxt='n', yaxt='n')
  text(10, 85, names(cm$byClass[1]), cex=1.2, font=2)
  text(10, 70, round(as.numeric(cm$byClass[1]), 3), cex=1.2)
  text(30, 85, names(cm$byClass[2]), cex=1.2, font=2)
  text(30, 70, round(as.numeric(cm$byClass[2]), 3), cex=1.2)
  text(50, 85, names(cm$byClass[5]), cex=1.2, font=2)
  text(50, 70, round(as.numeric(cm$byClass[5]), 3), cex=1.2)
  text(70, 85, names(cm$byClass[6]), cex=1.2, font=2)
  text(70, 70, round(as.numeric(cm$byClass[6]), 3), cex=1.2)
  text(90, 85, names(cm$byClass[7]), cex=1.2, font=2)
  text(90, 70, round(as.numeric(cm$byClass[7]), 3), cex=1.2)

  # add in the accuracy information 
  text(30, 35, names(cm$overall[1]), cex=1.5, font=2)
  text(30, 20, round(as.numeric(cm$overall[1]), 3), cex=1.4)
  text(70, 35, names(cm$overall[2]), cex=1.5, font=2)
  text(70, 20, round(as.numeric(cm$overall[2]), 3), cex=1.4)
}